In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from langdetect import detect
import morfeusz2
import unidecode

In [2]:
df = pd.read_excel("game_reviews.xlsx", index_col=0)

In [3]:
df

,review,voted_up
0,all,True
1,Battle Chasers: League of Legends 2.0,True
2,Ukończyłem już ponad połowę gry i jestem bardz...,True
3,Amazing. One of the best games I've played thi...,True
4,<3,True
...,...,...
74928,Spoko (=,True
74929,nice i alpha\n,True
74930,narazie bez błedów.\ngrafika obłedna\ngra sie ...,True
74931,8/10,True


In [9]:
df.iloc[2]['review']

'Ukończyłem już ponad połowę gry i jestem bardzo zadowolony z zakupu. Początek wyglądał nieco niemrawo, ale po dłuższej chwili gra bardzo zyskała. Klimatyczne i estetyczne lokacje, stosunkowo rozbudowany ale i zarazem bardzo przyjemny i prosty system rozwoju postaci (runy, ulepszanie umiejętności, zaklinanie przedmiotów itd) Walka turowa bardzo zyskuje przy bliższym poznaniu co tyczy się oczywiście dobrego dysponowania wszelkimi zasobami i atakami. Na początku trochę się jej obawiałem, ale po kilku godzinach gry wręcz ją bardzo lubię...\nCo najważniejsze fabuła jest dobra i można śmiało grać na najłatwiejszym poziomie, ciesząc się nią a zarazem wciąż bawiąc się swoją drużyną w walce. Ogólnie LoL ma teraz arcydobry okres. Bardzo polecam!'

In [4]:
class PreprocessingSteamReviews():
    def __init__(self, df_reviews):
        self.df_reviews = df_reviews
        
        self.remove_reviews_under_99_chars(99)
        self.df_reviews['review'] = self.df_reviews['review'].apply(self.remove_newlines_tabs)
        self.df_reviews['review'] = self.df_reviews['review'].apply(self.strip_html_tags)
        self.df_reviews['review'] = self.df_reviews['review'].apply(self.remove_whitespace)
        self.df_reviews['review'] = self.df_reviews['review'].apply(self.remove_non_alphanumeric_chracters)
        self.df_reviews['review'] = self.df_reviews['review'].apply(self.remove_links)
        self.remove_reviews_with_no_alphanumeric_items()
        self.remove_non_polish_reviews()
        self.lowercase_all()
        self.tokenize_all()
        self.remove_reviews_under_n_words(20)
        self.df_reviews['review'] = self.df_reviews['review'].apply(self.remove_polish_stopwords)
        
        self.morf = morfeusz2.Morfeusz()
        self.df_reviews['review'] = self.df_reviews['review'].apply(self.lemmatisation)
        self.df_reviews['review'] = self.df_reviews['review'].apply(self.remove_polish_stopwords)
        
        self.df_reviews['review'] = self.df_reviews['review'].apply(" ".join)
        self.df_reviews.drop(['len'], inplace=True, axis=1)
        self.df_reviews.drop(['len2'], inplace=True, axis=1)
        
        
    def remove_reviews_under_99_chars(self, n):
        self.df_reviews['len'] = self.df_reviews['review'].str.len()
        self.df_reviews = self.df_reviews[self.df_reviews['len'] > n]
        
    def remove_reviews_under_n_words(self, n):
        self.df_reviews['len2'] = self.df_reviews['review'].str.len()
        self.df_reviews = self.df_reviews[self.df_reviews['len2'] > n]
    
    def remove_newlines_tabs(self, text):
        Formatted_text = text.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ')
        return Formatted_text

    def strip_html_tags(self, text):
        soup = BeautifulSoup(text, "html.parser")
        stripped_text = soup.get_text(separator=" ")
        return stripped_text

    def remove_whitespace(self, text):
        pattern = re.compile(r'\s+') 
        Without_whitespace = re.sub(pattern, ' ', text)
        text = Without_whitespace.replace('?', ' ? ').replace(')', ') ')
        return text

    def remove_non_alphanumeric_chracters(self, text):
        regex = re.compile('[^a-zA-ZAaĄąBbCcĆćDdEeĘęFfGgHhIiJjKkLlŁłMmNnŃńOoÓóPpRrSsŚśTtUuWwYyZzŹźŻż ]')
        text = regex.sub('', text)
        return text
    
    def remove_reviews_with_no_alphanumeric_items(self):
        for row, data in self.df_reviews.T.iteritems():
            if not any(c.isalpha() for c in data['review']):
                self.df_reviews.drop([row], inplace=True)
                
    def remove_non_polish_reviews(self):
        for row, data in self.df_reviews.T.iteritems():
            if detect(data['review']) != 'pl':
                self.df_reviews.drop([row], inplace=True)
                
    def lowercase_all(self):
        self.df_reviews['review'] = self.df_reviews['review'].str.lower()
        
    def tokenize_all(self):
        self.df_reviews['review'] = self.df_reviews['review'].str.split()
    
    def remove_polish_stopwords(self, text):
        stopwords = []
        with open("polish.stopwords.txt", encoding = 'utf-8') as f:
            for line in f:
                stripped_line = line.strip()
                stopwords.append(stripped_line)
        words = [word for word in text if word.lower() not in stopwords]
        return words
    
    def lemmatisation(self, text):
        res = []
        for i in text:
            analysis = self.morf.analyse(i)
            x = analysis[0][2][1]
            x = x.split(':')[0].lower()
            res.append(x)
        return res
    
    def remove_links(self, text):
        remove_https = re.sub(r'http\S+', '', text)
        remove_com = re.sub(r"\ [A-Za-z]*\.com", " ", remove_https)
        remove_pl = re.sub(r"\ [A-Za-z]*\.pl", " ", remove_com)
        return remove_pl

In [5]:
pre = PreprocessingSteamReviews(df)
df_preprocessed = pre.df_reviews

C:\Users\user\AppData\Local\Temp/ipykernel_373236/107720559.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df_reviews['review'] = self.df_reviews['review'].apply(self.remove_newlines_tabs)
C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\bs4\__init__.py:431: MarkupResemblesLocatorWarning: "https://www.google.pl/maps/place/Mleczna+2,+43-436+G%C3%B3rki+Ma%C5%82e/@49.7615104,18.8474766,56m/data=!3m1!1e3!4m5!3m4!1s0x47141db60161f2b3:0xb341a26bf09415a5!8m2!3d49.7616348!4d18.8472092" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
C:\Users\user\AppData\Local\Programs\

In [6]:
df_preprocessed

,review,voted_up
2,ukończyć połowa zadowolić zakup początek wyglą...,True
12,gram league of legends lato miło koniec pograć...,True
16,witać wstęp powiedzieć nie fan league of legen...,True
19,super gierka zarówno fan lolo chcieć dowiedzie...,True
21,ciekawy rozgrywka nie dopracować pare bugow ud...,True
...,...,...
74911,witać grać wcześnie lost arka serwer ru opinia...,True
74920,wyglądać niesamowicie muzyka bajka rozgrywka p...,True
74922,gierka spoko dobry diablo iii sporo akcja sześ...,True
74926,halo polski scena growa pogrom yesieni pogrom ...,True


In [11]:
df_preprocessed.iloc[0]['review']

'ukończyć połowa zadowolić zakup początek wyglądać nieco niemrawa długi chwila zyskać klimatyczny estetyczny lokacja stosunkowo rozbudować zarazem przyjemny prosty system rozwój postać runy ulepszać umiejętność zaklinać przedmiot itd walka turowy zyskiwać bliski poznanie tycz oczywiście dobre dysponować wszelki zasoby atak początek trocha obawiać godzina wręcz lubić ważny fabuła dobra śmiać grać łatwy poziom cieszyć zarazem wciąż bawić swój drużyna walc ogólnie lol arcydobry okres polecać'

In [18]:
df_preprocessed[df_preprocessed['voted_up'] == False]

,review,voted_up
53,polska nie polecać raczy obsługa amazon grać g...,False
60,darmowy mało bug system kooperacja leż dangeon...,False
61,przyjemny człowiek chcieć przespacerować pobić...,False
66,wciągnąć godzina aktualizacja tracić wiara nap...,False
68,nie polecać główny konto zostać zbanować nie o...,False
...,...,...
74776,zajebiście cieszyć smite steamie przejść recen...,False
74797,grać człowiek wojownik kapłan asmodeusa lord d...,False
74803,dobry zalecać wstrzymać zakup pewien czas chwi...,False
74822,chcieć podkreślić pierwsza przygoda pathfinder...,False


In [19]:
df1 = df_preprocessed[df_preprocessed['voted_up'] == False]
len_f = len(df_preprocessed[df_preprocessed['voted_up'] == False])
print(len_f)

1829


In [20]:
df2 = df_preprocessed[df_preprocessed['voted_up'] == True][:len_f]

In [21]:
df_preprocessed = pd.concat([df1,df2])

In [22]:
df_preprocessed = df_preprocessed.sample(frac=1)

In [23]:
df_preprocessed.to_excel("game_reviews_preprocessed____.xlsx")